# Flow and transport problem - SPE10

In this tutorial we investigate the transport problem where the advective field is computed with a Darcy model.

Before creating the grid we import NumPy, the SciPy sparse library and PorePy.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

import os

cwd_folder = os.getcwd()
spe10_folder = cwd_folder + "/spe10/"

import sys

sys.path.insert(1, spe10_folder)

from spe10 import Spe10

We specify number of cells in each dimension and the physical size of the domain. Then we create a Cartesian grid and compute geometric properties such as face centers, cell volumes etc.

In [2]:
selected_layers = 35  # 3, 35

# Define the class with the corresponding layer(s)
spe10 = Spe10(selected_layers)
# For simplicity we extract the grid form the class spe10
sd = spe10.sd

# Read the permeability associated to the given layer(s)
perm_folder = spe10_folder + "/perm/"
spe10.read_perm(perm_folder)
perm_dict = spe10.perm_as_dict()

We declare the data for the Darcy problem.

In [3]:
# Permeability
perm = pp.SecondOrderTensor(
    kxx=perm_dict["kxx"], kyy=perm_dict["kyy"], kzz=perm_dict["kzz"]
)

# Boundary conditions
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
b_face_centers = sd.face_centers[:, b_faces]

# define outflow and inflow type boundary conditions, left and right boundary
outflow = np.isclose(b_face_centers[1, :], spe10.full_physdims[1])
inflow = np.isclose(b_face_centers[1, :], 0)

# define the labels and values for the boundary faces
labels = np.array(["neu"] * b_faces.size)
labels[np.logical_or(inflow, outflow)] = "dir"

bc_val = np.zeros(sd.num_faces)
bc_val[b_faces[inflow]] = 1e8

bc = pp.BoundaryCondition(sd, b_faces, labels)

We now set the data for the Darcy problem

In [4]:
flow_key = "flow"

# Collect all parameters in a dictionary
param = {pg.SECOND_ORDER_TENSOR: perm, "bc_values": bc_val, "bc": bc}
flow_data = pp.initialize_data({}, flow_key, param)

We now solve the Darcy problem by using the MPFA scheme.

In [5]:
# construct the lhr and rhs from the discretization of the diffusion operator
mpfa = pp.Mpfa(flow_key)
mpfa.discretize(sd, flow_data)
A, b = mpfa.assemble_matrix_rhs(sd, flow_data)

# solve the problem
cell_p = sps.linalg.spsolve(A, b)

# now data contains the discretization matrices build from MPFA
mat_discr = flow_data[pp.DISCRETIZATION_MATRICES][flow_key]

q = mat_discr["flux"] @ cell_p + mat_discr["bound_flux"] @ bc_val

And we export the corresponding solutions.

In [6]:
# to export the flux we consider the virtual version of the rt0 to work on any cell type
vrt0 = pg.VRT0(flow_key)

# compute an operator for evaluating the flux in the cell centers
proj = vrt0.eval_at_cell_centers(sd)

# construct the P0 flux reconstruction, no reshape is needed
cell_q = proj @ q

save = pp.Exporter(sd, "sol_p", folder_name="ex5")

data_to_export = [
    ("kxx", np.log10(perm_dict["kxx"])),
    ("kyy", np.log10(perm_dict["kyy"])),
    ("kzz", np.log10(perm_dict["kzz"])),
    ("cell_p", cell_p),
    ("cell_q", cell_q),
]
save.write_vtu(data_to_export)

We now consider the transport problem where now the advective field is the one computed from the Darcy problem. First we set the data.

In [7]:
# Transport problem
transport_key = "transport"
delta_t = 0.001
num_steps = 40

# Set in the data file the flux
bc_val = np.zeros(sd.num_faces)
bc_val[b_faces[inflow]] = 1

# Collect all parameters in a dictionary
param = {"darcy_flux": q, "bc_values": bc_val, "bc": bc}
transport_data = pp.initialize_data({}, transport_key, param)

As done in the other cases, we now construct the upwind matrix and the mass matrix.

In [8]:
# create the upwind and mass matrices
upwind = pp.Upwind(transport_key)

# discretize and get the matrices
upwind.discretize(sd, transport_data)

U, b_upwind = upwind.assemble_matrix_rhs(sd, transport_data)
M = sps.diags(sd.cell_volumes)

Finally, by using the implicit Euler we compute the concentration that is transported in the porous medium

In [9]:
# Initial condition and exporter
c = np.zeros(sd.num_cells)
save = pp.Exporter(sd, "sol_c", folder_name="ex5")
save.write_vtu([("conc", c)], time_step=0)

# IE with LU factorization
S = M + delta_t * U
lu = sps.linalg.splu(S.tocsc())
for i in np.arange(num_steps):
    c = lu.solve(M @ c - delta_t * b_upwind)
    print(np.amin(c), np.amax(c))
    save.write_vtu([("conc", c)], time_step=i + 1)

# export the main pvd file
time = np.arange((num_steps + 1)) * delta_t
save.write_pvd(time)

5.402641987709054e-25 0.982276025846727
8.351687999381401e-24 0.9996858607401834
6.787036078284859e-23 0.9999944322038473
3.8605403165518315e-22 0.9999999013164935
1.726681481536223e-21 0.9999999982509048
6.468244747944901e-21 0.9999999999689678
2.111027360223641e-20 0.9999999999994187
6.165620870895617e-20 1.0000000000000462
1.6429132629774008e-19 1.0000000000000608
4.0521914625215066e-19 1.0000000000001819
9.355372397122227e-19 1.000000000000525
2.039749542028547e-18 1.0000000000006797
4.230059004017441e-18 1.0000000000007023
8.393096803967797e-18 1.0000000000007039
1.6011358544075653e-17 1.0000000000007039
2.948846892260345e-17 1.0000000000007039
5.261561631657338e-17 1.0000000000007039
9.122621472297151e-17 1.0000000000007039
1.540971833837295e-16 1.0000000000007039
2.541682971636452e-16 1.0000000000007039
4.1016595737083705e-16 1.0000000000009042
6.487345572141777e-16 1.000000000002001
1.0071974516653658e-15 1.0000000000022378
1.53709045985522e-15 1.000000000363777
2.3086338890761

In [10]:
# Consistency check
assert np.isclose(np.linalg.norm(c), 103.27075674401456)